This try to install the required libraries in your Jupyter server. Run once.

In [ ]:
import sys
!{sys.executable} -m pip install pyserial
!{sys.executable} -m pip install pyvisa-py
!{sys.executable} -m pip install pyvisa
!{sys.executable} -m pip install pandas
!{sys.executable} -m pip install matplotlib

Import the library and select the python visa backend

The command list_resources() shows a list of connected instruments. you can use the different ports to be used in the open_resource() call of the next cell.

In [ ]:
import pyvisa
import time

rm = pyvisa.ResourceManager('@py')
rm.list_resources()

This cell opens the specified instrument and shows its identification. Use the identification to check that you are speaking with the correct instrument.

In [ ]:
dmm = rm.open_resource('ASRL/dev/ttyUSB1')
#dmm.write("*RST")
#time.sleep(10)
print(dmm.query("*IDN?")) 

The next cell reads the oven values and combine it with the temperature from digital multimeter (dmm)

Update te variable serialOven to the port where your oven is connected.

In [ ]:
import serial
import re
from datetime import datetime
import pandas as pd

serialOven = '/dev/ttyUSB3'

ovenMeasureExpression = re.compile(r"^\s*(\d*.\d*),\s*(\d*.\d*),\s*(\d*.\d*),\s*(\d*.\d*),\s*(\d*.\d*),\s*(\d*),\s*(\d*.\d*),\s*(\d*),\s*(\d*),\s*(\d*.\d*),\s*([a-zA-Z-]*)")
ovenEndExpression = re.compile(r"([Ee]nd bake)|([Rr]eflow done)")

measures = []

with serial.Serial(serialOven, 115200, timeout=5, bytesize=8, parity=serial.PARITY_NONE, stopbits=1) as ser:
    while True: 
        ovenRead = str(ser.readline(), 'ascii')
        match = re.search(ovenMeasureExpression, ovenRead)
        if match:
            dmmTemp = dmm.query("READ?")
            measures.append([datetime.now(), match.group(1), match.group(2), match.group(3), match.group(6), match.group(7), match.group(8), match.group(9), match.group(10), match.group(11), dmmTemp])
        elif ovenEndExpression.match(ovenRead):
            break
        else:
            print (ovenRead)

dfMeasures = pd.DataFrame(measures, columns = ['timestamp', 'ovenTime', 'temp0', 'temp1', 'ovenSet', 'actual', 'heat', 'fan', 'coldJunction', 'mode', 'dmmTemp'])
dfMeasures['ovenTime'] = pd.to_numeric(dfMeasures['ovenTime'],errors='coerce')
dfMeasures['temp0'] = pd.to_numeric(dfMeasures['temp0'],errors='coerce')
dfMeasures['temp1'] = pd.to_numeric(dfMeasures['temp1'],errors='coerce')
dfMeasures['ovenSet'] = pd.to_numeric(dfMeasures['ovenSet'],errors='coerce')
dfMeasures['actual'] = pd.to_numeric(dfMeasures['actual'],errors='coerce')
dfMeasures['heat'] = pd.to_numeric(dfMeasures['heat'],errors='coerce')
dfMeasures['fan'] = pd.to_numeric(dfMeasures['fan'],errors='coerce')
dfMeasures['coldJunction'] = pd.to_numeric(dfMeasures['coldJunction'],errors='coerce')
dfMeasures['dmmTemp'] = pd.to_numeric(dfMeasures['dmmTemp'],errors='coerce')

Plot measured data

In [ ]:
import matplotlib.pyplot as plt

fig= plt.figure(figsize=(25,12))
axes= fig.add_axes([0.1,0.1,0.8,0.8])

plt.plot( 'timestamp', 'ovenSet', data=dfMeasures, marker='', color='red', linewidth=4, label='Set')
axes.plot( 'timestamp', 'actual', data=dfMeasures, marker='', color='olive', linewidth=4, label='Oven')
axes.plot( 'timestamp', 'dmmTemp', data=dfMeasures, marker='', color='skyblue', linewidth=4, label='dmm')

axes.legend()

plt.show()
